# Testing

The developed classes need some thorough testing before we run the whole experiment. It would be very bad if the script breaks in the middle of tuning. This notebook lists some situations that should be tested and explains how to do so.

The tests can be performed by evaluating the objective function for a specified set of parameter values. Every class has a method <i>objfunction</i>, which takes a list of values as input.

In [1]:
import os
os.chdir('..')
import helpers
import models
from models import OneShotCNN, OneShotTransferCNN, OneShotAutoencoder, OneShotSiameseNetwork

C:\Users\s100385\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\s100385\Documents\JADS Working Files\Research Paper One-shot\Code\OneShotDL\models.py:1086: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(not np.any(np.equal(classes1[:batch_size//2], classes2[:batch_size//2])), "Error in class selection")


In [2]:
# initialize with logging and verbose
model = OneShotCNN(log=True, verbose=1)
# choose lower bound of model as example (THIS SHOULD BE DONE MANUALLY TO PERFORM THE TESTS)
x = model.xlow
print(x)

[1.e-04 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 2.e+00 0.e+00 0.e+00
 3.e+00 4.e+00 5.e+00 0.e+00 0.e+00 5.e+01]


As you can see, x is just a list of numbers. This can be put in the objective function

In [ ]:
# test for given x
model.objfunction(x)

The <i>hyper_map</i> attribute of the model can be used to get the index of a hyperparameter in X. For example, the learning rate is the first entry of X (index 0).

In [ ]:
model.hyper_map['learning_rate']

to see the full list of available hyperparameters (and their order), you can print the <i>hyperparams</i> attribute:

In [ ]:
model.hyperparams

# OneShotCNN tests

For the simplest class, the OneShotCNN class, we should test the following:
- The early stopping procedure in combination with logging. Do this by training a very bad model (setting RMSProp's learning rate to 0.1 should do the trick) and have log=True on initialization.

In [ ]:
test = OneShotCNN(log=True, verbose=0)
x = test.xlow
x[test.hyper_map['learning_rate']] = 0.000000000001
test.objfunction(x)

##### Works as intended

# OneShotTransferCNN tests

For the simplest class, the OneShotCNN class, we should test the following:
- The early stopping procedure in combination with logging. Do this by training a very bad model (setting RMSProp's learning rate to 0.1 should do the trick) and have log=True on initialization.
- Having 'num_fixed_layers' larger than the total number of layers in the model.
- ..

In [ ]:
transfermodel = OneShotTransferCNN(log=True, verbose=1)

In [ ]:
x = transfermodel.xlow
# set learning rates to too very bad values
x[0] = 0.001 # big enough to learn nothing
x[1] = 0.001 # don't let it learn much in fine tuning
# set fixed layers to value bigger than the total number of layers
x[transfermodel.hyper_map['num_fixed_layers']] = 9
x[transfermodel.hyper_map['epochs']] = 1
x[transfermodel.hyper_map['finetune_epochs']] = 3

In [ ]:
transfermodel.objfunction(x)

##### Outcomes
- Early stopping works as intended and logging is not affected by it.
- The big number of fixed layers results in the classifying layer also being fixed. This way, no fine tuning occurs. <strong>Problem fixed at March 6, 2018.</strong>.

# OneShotAutoencoder tests

For the autoencoder class
- General objective evaulation
- Tuning procedure
- Whether weights are frozen and reinitialized properly
- Positioning of max pools and upsampling layers

In [ ]:
oneshotAE = OneShotAutoencoder(log=True, folds=3, verbose=1)

In [ ]:
#oneshotAE.objfunction(x)

In [ ]:
oneshotAE.tune_with_HORD(100)

##### Outcomes
- Objective evaluation works
- Tuning works
- Freezing layers works properly for the case when all layers should be frozen except the classifier. <strong>Other values need more testing</strong>
- Max pools and upsampling layers seem to position just fine.

# Siamese Network tests

In [1]:
import os
os.chdir('..')
import helpers
import models
from models import OneShotCNN, OneShotTransferCNN, OneShotAutoencoder, OneShotSiameseNetwork
from copy import copy

C:\Users\s100385\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\s100385\Documents\JADS Working Files\Research Paper One-shot\Code\OneShotDL\models.py:1092: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(not np.any(np.equal(classes1[:batch_size//2], classes2[:batch_size//2])), "Error in class selection")


In [2]:
siamesenet = OneShotSiameseNetwork()

In [3]:
x = copy(siamesenet.xlow)

In [4]:
siamesenet.objfunction(x)

-------------
Experiment 1.
-------------
learning_rate: 0.0001
dropout_rate1: 0.0
dropout_rate2: 0.0
width_shift: 0.0
height_shift: 0.0
shear: 0.0
zoom: 0.0
num_conv_layers: 2.0
num_dense_layers: 0.0
num_maxpools: 0.0
neurons_first_conv: 3.0
neurons_remaining_conv: 4.0
neurons_dense: 5.0
neurons_final: 5.0
rotation: 0.0
horizontal_flip: 0.0
epochs: 3.0
-------------
Cross validating in 10 folds..


C:\Users\s100385\Documents\JADS Working Files\Research Paper One-shot\Code\OneShotDL\models.py:1047: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  combined = merge([encoded_left, encoded_right], mode = self.L1_distance, output_shape = lambda x: x[0])
C:\Users\s100385\Anaconda3\lib\site-packages\keras\legacy\layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
C:\Users\s100385\Documents\JADS Working Files\Research Paper One-shot\Code\OneShotDL\models.py:1049: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  model = Model(input=[left_input, right_input], output=prediction)


fit 1:
test accuracy: 55.04%.
fit 2:
test accuracy: 50.74%.
fit 3:
test accuracy: 49.98%.
fit 4:
test accuracy: 63.92%.
fit 5:
test accuracy: 40.58%.
fit 6:
test accuracy: 20.0%.
fit 7:
test accuracy: 48.98%.
fit 8:
test accuracy: 45.54%.
fit 9:
test accuracy: 47.6%.
fit 10:
test accuracy: 62.6%.
Scores: [0.5504, 0.5074, 0.4998, 0.6392, 0.4058, 0.2, 0.4898, 0.4554, 0.476, 0.626].
Mean: 48.5%. Standard deviation: 11.71%


-0.48498